In [33]:
# Area code metadata
AREA_CODES = {
    "C4266": {
        "code": "C4266",
        "name": "Seattle-Tacoma-Bellevue",
        "state": "WA",
        "region": "Seattle-Tacoma-Bellevue, WA MSA",
        "type": "MSA"
    },
    "C3890": {
        "code": "C3890",
        "name": "Portland-Vancouver-Hillsboro",
        "state": "OR-WA",
        "region": "Portland-Vancouver-Hillsboro, OR-WA MSA",
        "type": "MSA"
    }
}


In [ ]:
# Example: load a QCEW area CSV using the cache utilities
from bls_housing import load_area_df, get_cached_path #, fetch_area_csv

# areas = AREA_CODES["C4266"]["code"] # Seattle-Tacoma-Bellevue MSA
years = ["2020","2021","2022","2023","2024","2025"]
qtr = "1"
# Load DataFrame (uses cache if available)
# get total quarterly wages for MSA for each year and store in total_wages dict
for code in AREA_CODES:
    total_wages = {}

    area = AREA_CODES[code]["code"]
    # print(f"Area: {AREA_CODES[code]['name']} ({area})")

    for year in years:
        df = load_area_df(area, year, qtr)
        # print(f"Loaded rows: {len(df)}")
        # Show cached path (or None)
        cached = get_cached_path(area, year, qtr)
        # print("Cached file:", cached)
        msa = df[df.get('agglvl_code') == 40]
        total_wages[year] = msa['total_qtrly_wages'][0]
        print(f"{total_wages[year]}")
        print(f"Year: {year}, Total Wages: {total_wages[year]:,.2f}")

    for i in range(1, len(years)):
        year = years[i]
        prev_year = years[i-1]
        change = ((total_wages[year] - total_wages[prev_year]) / total_wages[prev_year]) * 100
        print(f"Year: {year}, Total Wages: {total_wages[year]:,.2f}, Change from {prev_year}: {change:.2f}%")   
    


45200515733
Year: 2020, Total Wages: 45,200,515,733.00
46310344060
Year: 2021, Total Wages: 46,310,344,060.00
50669607121
Year: 2022, Total Wages: 50,669,607,121.00
55513395217
Year: 2023, Total Wages: 55,513,395,217.00
58911332927
Year: 2024, Total Wages: 58,911,332,927.00
62262553595
Year: 2025, Total Wages: 62,262,553,595.00
Year: 2021, Total Wages: 46,310,344,060.00, Change from 2020: 2.46%
Year: 2022, Total Wages: 50,669,607,121.00, Change from 2021: 9.41%
Year: 2023, Total Wages: 55,513,395,217.00, Change from 2022: 9.56%
Year: 2024, Total Wages: 58,911,332,927.00, Change from 2023: 6.12%
Year: 2025, Total Wages: 62,262,553,595.00, Change from 2024: 5.69%
19780516586
Year: 2020, Total Wages: 19,780,516,586.00
19545962532
Year: 2021, Total Wages: 19,545,962,532.00
21956438929
Year: 2022, Total Wages: 21,956,438,929.00
23674348879
Year: 2023, Total Wages: 23,674,348,879.00
24212290614
Year: 2024, Total Wages: 24,212,290,614.00
24657399584
Year: 2025, Total Wages: 24,657,399,584.00


In [ ]:
# get year over year percent change in total wages    



TypeError: unsupported format string passed to Series.__format__

In [ ]:
# # Quick peek and filter to MSA totals (agglvl_code == 40)
# print(df.head())
# msa = df[df.get('agglvl_code') == 40]
# print(f"MSA rows: {len(msa)}")
# print(msa.head())

In [ ]:
# import requests
#import json
#import pandas as pd
# import numpy as np
'''C4266 Seattle-Tacoma-Bellevue, WA MSA QCEW Area Code
C3890 Portland-Vancouver-Hillsboro, OR-WA MSA QCEW Area Code'''
# headers = {'Content-Type': 'application/json'}

# import urllib.request
# import urllib
# def qcewCreateDataRows(csv):
#     dataRows = []
#     try: dataLines = csv.decode().split('\r\n')
#     except er: dataLines = csv.split('\r\n')
#     for row in dataLines:
#         dataRows.append(row.split(','))
#     return dataRows

# def qcewGetAreaData(year,qtr,area):
#     urlPath = "http://data.bls.gov/cew/data/api/[YEAR]/[QTR]/area/[AREA].csv"
#     urlPath = urlPath.replace("[YEAR]",year)
#     urlPath = urlPath.replace("[QTR]",qtr.lower())
#     urlPath = urlPath.replace("[AREA]",area.upper())
#     httpStream = urllib.request.urlopen(urlPath)
#     csv = httpStream.read()
#     httpStream.close()
#     return qcewCreateDataRows(csv)

# def qcewGetAreaUrl(year,qtr,area):
#     urlPath = "https://data.bls.gov/cew/data/api/[YEAR]/[QTR]/area/[AREA].csv"
#     urlPath = urlPath.replace("[YEAR]",year)
#     urlPath = urlPath.replace("[QTR]",qtr.lower())
#     urlPath = urlPath.replace("[AREA]",area.upper())
#     return urlPath

# URLPATH = qcewGetAreaUrl("2025", "1", "C4266")
#httpStream = urllib.request.urlopen(URLPATH)
#httpStreamData = httpStream.read()



In [ ]:
# print(httpStreamData)
# pdf = pd.read_csv(URLPATH)
# print(pdf[0])
# print(pdf)
# httpStreamDataRows = qcewCreateDataRows(httpStreamData)

# for row in httpStreamDataRows:
#     print(row)

In [ ]:
# important values, agglvl_code 40 is MSA Total covered, own_code 0 is total covered, industry_code 10 is all industries, size_code 0 is All establishment sizes.
# filtered_df = pdf[pdf['agglvl_code'] == 40]
# print(filtered_df)